In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Coronary_artery_disease"
cohort = "GSE120774"

# Input paths
in_trait_dir = "../../input/GEO/Coronary_artery_disease"
in_cohort_dir = "../../input/GEO/Coronary_artery_disease/GSE120774"

# Output paths
out_data_file = "../../output/preprocess/Coronary_artery_disease/GSE120774.csv"
out_gene_data_file = "../../output/preprocess/Coronary_artery_disease/gene_data/GSE120774.csv"
out_clinical_data_file = "../../output/preprocess/Coronary_artery_disease/clinical_data/GSE120774.csv"
json_path = "../../output/preprocess/Coronary_artery_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression Data from epicardial  (EAT) and subcutaneous adipose tissue (SAT) in patients with coronary artery disease"
!Series_summary	"We took samples of subcutaneous adipose tissue from the sternum (SAT) and epicardial adipose tissue (EAT) from a site adjacent to the right coronary artery in cases with coronary disease and controls without coronary disease."
!Series_summary	"Cases had significant coronary disease and were undergoing coronary artery bypass surgery.  Controls all had coronary angiograms and did not have significant coronary disease."
!Series_overall_design	"Case control study"
Sample Characteristics Dictionary:
{0: ['tissue: subcutaneous fat', 'tissue: epicardial fat'], 1: ['gender: M', 'gender: F'], 2: ['disease: None', 'disease: coronary disease']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this study collected tissue samples (EAT and SAT) from patients,
# which suggests gene expression data is likely available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait (Coronary artery disease) - available in row 2 with unique values "disease: None" and "disease: coronary disease"
trait_row = 2

# Gender - available in row 1 with unique values "gender: M" and "gender: F"
gender_row = 1  

# Age - not provided in the sample characteristics
age_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (0 for controls, 1 for cases)"""
    if not isinstance(value, str):
        return None
    value = value.lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "none" in value or "control" in value or "without" in value:
        return 0
    elif "coronary disease" in value or "cad" in value:
        return 1
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    value = value.lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value == "f" or "female" in value:
        return 0
    elif value == "m" or "male" in value:
        return 1
    return None

# Age conversion not needed as age data is not available
convert_age = None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial filtering on usability and save information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract and save clinical features
if is_trait_available:
    try:
        # Extract clinical features - using the clinical_data variable that should be 
        # available from previous steps
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Variable expected to be available from previous steps
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        # Preview the dataframe
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save the extracted clinical features
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Saved clinical data to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")


Preview of selected clinical features:
{'GSM3415000': [0.0, 1.0], 'GSM3415001': [0.0, 1.0], 'GSM3415002': [0.0, 1.0], 'GSM3415003': [0.0, 1.0], 'GSM3415004': [0.0, 0.0], 'GSM3415005': [0.0, 0.0], 'GSM3415006': [0.0, 1.0], 'GSM3415007': [0.0, 0.0], 'GSM3415008': [0.0, 0.0], 'GSM3415009': [1.0, 1.0], 'GSM3415010': [1.0, 1.0], 'GSM3415011': [1.0, 1.0], 'GSM3415012': [1.0, 0.0], 'GSM3415013': [1.0, 1.0], 'GSM3415014': [1.0, 0.0], 'GSM3415015': [1.0, 1.0], 'GSM3415016': [1.0, 0.0], 'GSM3415017': [0.0, 0.0], 'GSM3415018': [0.0, 1.0], 'GSM3415019': [0.0, 1.0], 'GSM3415020': [0.0, 0.0], 'GSM3415021': [0.0, 0.0], 'GSM3415022': [0.0, 0.0], 'GSM3415023': [0.0, 1.0], 'GSM3415024': [0.0, 1.0], 'GSM3415025': [0.0, 1.0], 'GSM3415026': [0.0, 0.0], 'GSM3415027': [1.0, 1.0], 'GSM3415028': [1.0, 1.0], 'GSM3415029': [1.0, 1.0], 'GSM3415030': [1.0, 1.0], 'GSM3415031': [1.0, 0.0], 'GSM3415032': [1.0, 0.0], 'GSM3415033': [1.0, 1.0], 'GSM3415034': [1.0, 1.0], 'GSM3415035': [1.0, 0.0]}
Saved clinical data to .

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Coronary_artery_disease/GSE120774/GSE120774_family.soft.gz
Matrix file: ../../input/GEO/Coronary_artery_disease/GSE120774/GSE120774_series_matrix.txt.gz
Found the matrix table marker at line 63
Gene data shape: (33297, 36)
First 20 gene/probe identifiers:
['7892501', '7892502', '7892503', '7892504', '7892505', '7892506', '7892507', '7892508', '7892509', '7892510', '7892511', '7892512', '7892513', '7892514', '7892515', '7892516', '7892517', '7892518', '7892519', '7892520']


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be probe IDs (numeric identifiers) rather than human gene symbols
# These are likely microarray probe IDs which need to be mapped to gene symbols

# Based on the format (7-digit numeric IDs), these appear to be Affymetrix or Illumina microarray probe IDs
# They need to be mapped to official gene symbols for meaningful biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examining the gene_assignment column which appears to contain gene symbol information
print("\nExamining 'gene_assignment' column examples:")
if 'gene_assignment' in gene_annotation.columns:
    # Display a few examples of the gene_assignment column to understand its format
    gene_samples = gene_annotation['gene_assignment'].head(5).tolist()
    for i, sample in enumerate(gene_samples):
        print(f"Example {i+1}: {sample[:200]}..." if isinstance(sample, str) and len(sample) > 200 else f"Example {i+1}: {sample}")
    
    # Check the quality and completeness of the gene_assignment column
    non_null_assignments = gene_annotation['gene_assignment'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene assignment column completeness: {non_null_assignments}/{total_rows} rows ({non_null_assignments/total_rows:.2%})")
    
    # Check for probe IDs without gene assignments (typically '---' entries)
    missing_assignments = gene_annotation[gene_annotation['gene_assignment'] == '---'].shape[0]
    print(f"Probes without gene assignments: {missing_assignments}/{total_rows} rows ({missing_assignments/total_rows:.2%})")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs (e.g., 7896736)")
    print("- 'gene_assignment': Contains gene information that needs parsing to extract gene symbols")
else:
    print("Error: 'gene_assignment' column not found in annotation data.")
    print("Available columns:", gene_annotation.columns.tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008'], 'seqname': ['chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091'], 'RANGE_STOP': ['54936', '63887', '70008'], 'total_probes': [7.0, 31.0, 24.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST00000492842 // OR4G11P // olfactory receptor, family 4, subfamily G, member 11 pseudogene // --- // --- /// ENST00000588632 // OR4G1P // olfactory receptor, family 4, subf

### Step 6: Gene Identifier Mapping

In [7]:
# 1. From the output of previous steps, we need to map the numeric probe IDs to gene symbols
# The gene annotation contains 'ID' column with probe IDs and 'gene_assignment' column with gene symbols and other info

# Load gene annotation and expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)
gene_data_raw = get_genetic_data(matrix_file)

# 2. Extract and process the mapping between probe IDs and gene symbols 
# Create a custom function to extract gene symbols from the gene_assignment string
def extract_gene_symbols_from_assignment(assignment_str):
    if not isinstance(assignment_str, str) or assignment_str == '---':
        return []
    
    # The format appears to be: RefSeq_ID // GENE_SYMBOL // description // location // ID
    # Multiple genes are separated by ///
    gene_symbols = []
    gene_entries = assignment_str.split('///')
    
    for entry in gene_entries:
        parts = entry.strip().split('//')
        if len(parts) >= 2:
            gene_symbol = parts[1].strip()
            # Some entries might have multiple symbols or additional info
            if gene_symbol and gene_symbol != '---':
                # Get the actual gene symbol without additional text
                # The gene symbols appear to be standard symbols like OR4F4
                symbols = extract_human_gene_symbols(gene_symbol)
                gene_symbols.extend(symbols)
    
    return list(set(gene_symbols))  # Remove duplicates

# Apply the function to create a mapping dataframe
gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(extract_gene_symbols_from_assignment)

# Print some examples to verify extraction
print("Example gene symbol extractions:")
for i, (assignment, symbols) in enumerate(zip(gene_annotation['gene_assignment'].head(5), gene_annotation['Gene'].head(5))):
    print(f"Example {i+1}:")
    print(f"Original: {assignment[:100]}..." if isinstance(assignment, str) and len(str(assignment)) > 100 else f"Original: {assignment}")
    print(f"Extracted symbols: {symbols}")

# Create mapping dataframe
mapping_df = gene_annotation[['ID', 'Gene']].copy()

# Keep only entries with at least one gene symbol
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]
print(f"\nTotal probes with valid gene symbols: {len(mapping_df)}")

# 3. Convert probe-level measurements to gene expression using apply_gene_mapping
# Instead of directly calling the function, we'll manually do the steps to debug
try:
    # Apply the mapping
    gene_data = apply_gene_mapping(gene_data_raw, mapping_df)
    
    # Print mapping statistics
    print("\nGene mapping information:")
    print(f"Total probes in expression data: {gene_data_raw.shape[0]}")
    print(f"Probes with gene mappings: {len(mapping_df)}")
    print(f"Unique genes after mapping: {gene_data.shape[0]}")
    
    # If mapping still fails, try a direct approach for urgent debugging
    if gene_data.shape[0] == 0:
        print("\nWARNING: Standard mapping produced 0 genes. Attempting alternate approach...")
        
        # Fallback approach: Create a simpler mapping using only unique first symbols
        simplified_mapping = gene_annotation[['ID']].copy()
        simplified_mapping['Gene'] = gene_annotation['gene_assignment'].apply(
            lambda x: extract_human_gene_symbols(str(x))[0] if isinstance(x, str) and len(extract_human_gene_symbols(str(x))) > 0 else None
        )
        simplified_mapping = simplified_mapping.dropna()
        print(f"Simplified mapping produced {len(simplified_mapping)} probe-to-gene mappings")
        
        # Apply simplified mapping
        gene_data = apply_gene_mapping(gene_data_raw, simplified_mapping)
        print(f"Simplified mapping produced {gene_data.shape[0]} genes")
    
    # Normalize gene symbols to ensure consistency
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Preview the final gene expression data
    print("\nPreview of gene expression data:")
    print(preview_df(gene_data, n=5))
    
    # Save the processed gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"\nSaved gene expression data to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    
    # If an error occurs, try a minimal approach to extract at least some data
    print("\nAttempting minimal approach to extract gene data...")
    
    # Create a direct mapping using a more lenient approach
    direct_mapping = []
    for idx, row in gene_annotation.iterrows():
        gene_id = row['ID']
        if isinstance(row['gene_assignment'], str):
            # Extract gene symbols using regex
            genes = re.findall(r'//\s+([A-Z0-9]+)\s+//', row['gene_assignment'])
            if genes:
                for gene in genes:
                    direct_mapping.append({'ID': gene_id, 'Gene': gene})
    
    if direct_mapping:
        direct_mapping_df = pd.DataFrame(direct_mapping)
        
        # Apply direct mapping
        gene_data = apply_gene_mapping(gene_data_raw, direct_mapping_df)
        
        # Normalize and save
        gene_data = normalize_gene_symbols_in_index(gene_data)
        gene_data.to_csv(out_gene_data_file)
        print(f"Minimal approach produced {gene_data.shape[0]} genes")
        print(f"Saved gene expression data to {out_gene_data_file}")
    else:
        print("Failed to extract gene data with all approaches. Cannot proceed with gene mapping.")


Example gene symbol extractions:
Example 1:
Original: ---
Extracted symbols: []
Example 2:
Original: ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- /...
Extracted symbols: ['OR4G1P', 'OR4G2P', 'OR4G11P']
Example 3:
Original: NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 ///...
Extracted symbols: ['OR4F4', 'OR4F5', 'OR4F17']
Example 4:
Original: NR_024437 // LOC728323 // uncharacterized LOC728323 // 2q37.3 // 728323 /// XM_006711854 // LOC10106...
Extracted symbols: ['PCMTD2', 'SEPT14']
Example 5:
Original: NM_001005221 // OR4F29 // olfactory receptor, family 4, subfamily F, member 29 // 1p36.33 // 729759 ...
Extracted symbols: ['OR4F28P', 'OR4F16', 'OR4F29', 'OR4F8P', 'OR4F7P', 'OR4F3', 'OR4F2P', 'OR4F1P', 'OR4F21']



Total probes with valid gene symbols: 24707

Gene mapping information:
Total probes in expression data: 33297
Probes with gene mappings: 24707
Unique genes after mapping: 0



Simplified mapping produced 25127 probe-to-gene mappings
Simplified mapping produced 23139 genes
After normalization: 22343 genes

Preview of gene expression data:
{'GSM3415000': [6.95774, 5.6657, 12.0967, 5.64055, 5.23009], 'GSM3415001': [7.61407, 5.90698, 11.7109, 6.19953, 5.92731], 'GSM3415002': [7.21409, 5.6717, 11.87, 6.11775, 5.49881], 'GSM3415003': [7.19709, 5.75878, 11.8535, 5.93999, 5.77457], 'GSM3415004': [7.12083, 5.69981, 11.9852, 5.85591, 5.55674], 'GSM3415005': [7.30062, 5.77384, 11.8517, 5.67189, 5.69931], 'GSM3415006': [7.21898, 6.0316, 11.7973, 5.74427, 5.64976], 'GSM3415007': [6.90321, 5.56953, 11.4465, 5.69886, 5.55623], 'GSM3415008': [7.16062, 5.81981, 11.7353, 5.72275, 5.76073], 'GSM3415009': [7.15482, 5.74577, 11.8572, 5.71592, 5.62793], 'GSM3415010': [7.17004, 5.74897, 11.6864, 5.74791, 6.06825], 'GSM3415011': [7.26014, 5.73882, 11.7929, 5.69294, 5.59686], 'GSM3415012': [7.21029, 5.66763, 11.8685, 5.52236, 5.54461], 'GSM3415013': [7.03885, 5.43717, 11.9798, 5.582


Saved gene expression data to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE120774.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Attempt to load gene data and handle possible issues with normalization
try:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Debug the trait variable and data structure
    print(f"Debug - trait variable: {trait} (type: {type(trait)})")
    
    # Check if gene_data (from previous step) has any content
    if 'gene_data' not in locals() or gene_data.shape[0] == 0:
        print("WARNING: Gene data is empty after normalization in previous step.")
        print("This appears to be miRNA data rather than gene expression data.")
        
        # Since gene_data is empty, set gene_available to False
        is_gene_available = False
        
        # Create an empty dataframe for metadata purposes
        empty_df = pd.DataFrame()
        
        # Log information about this dataset for future reference
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=True,  # Based on previous steps
            is_biased=True,  # Consider it biased as we can't use it
            df=empty_df,
            note="Dataset appears to contain miRNA data rather than gene expression data. Gene symbols could not be normalized."
        )
        
        print("Dataset marked as unusable due to lack of valid gene expression data.")
    else:
        # If gene_data is not empty, proceed with normalization and linking
        print(f"Gene data shape before normalization: {gene_data.shape}")
        
        # Save the gene data we have, even if it's already normalized
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene data saved to {out_gene_data_file}")
        
        # Attempt to link clinical and gene data
        try:
            # Load clinical data
            clinical_features = pd.read_csv(out_clinical_data_file)
            print(f"Loaded clinical data shape: {clinical_features.shape}")
            print(f"Clinical data columns: {clinical_features.columns.tolist()}")
            
            # Set the index for clinical features if needed
            if 'Unnamed: 0' in clinical_features.columns:
                clinical_features = clinical_features.set_index('Unnamed: 0')
            
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            print(f"Linked data columns: {linked_data.columns[:10].tolist()}...")  # Show first 10 columns
            
            # Check which columns are available for trait
            if trait in linked_data.columns:
                trait_column = trait
            else:
                # Since clinical data only has 2 rows, and one is likely Gender, the other is the trait
                # Find column that's not Gender
                possible_trait_columns = [col for col in linked_data.columns if col != 'Gender']
                trait_column = possible_trait_columns[0] if possible_trait_columns else linked_data.columns[0]
            
            print(f"Using trait column name: {trait_column}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait_column)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_column)
                
                # Validate data quality and save cohort info
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=True,
                    is_biased=is_biased,
                    df=linked_data,
                    note="Successfully processed gene expression data for coronary artery disease."
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=True,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        except Exception as inner_e:
            print(f"Error in linking or processing data: {inner_e}")
            import traceback
            traceback.print_exc()
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note=f"Error during linking or processing: {str(inner_e)}"
            )
except Exception as e:
    print(f"Error in data processing: {e}")
    import traceback
    traceback.print_exc()
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # Consider gene data unavailable if we had an error
        is_trait_available=True,  # Based on previous steps
        is_biased=True,  # Consider it biased as we can't use it
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note=f"Error during normalization or linking: {str(e)}"
    )

Debug - trait variable: Coronary_artery_disease (type: <class 'str'>)
Gene data shape before normalization: (22343, 36)


Gene data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE120774.csv
Loaded clinical data shape: (2, 36)
Clinical data columns: ['GSM3415000', 'GSM3415001', 'GSM3415002', 'GSM3415003', 'GSM3415004', 'GSM3415005', 'GSM3415006', 'GSM3415007', 'GSM3415008', 'GSM3415009', 'GSM3415010', 'GSM3415011', 'GSM3415012', 'GSM3415013', 'GSM3415014', 'GSM3415015', 'GSM3415016', 'GSM3415017', 'GSM3415018', 'GSM3415019', 'GSM3415020', 'GSM3415021', 'GSM3415022', 'GSM3415023', 'GSM3415024', 'GSM3415025', 'GSM3415026', 'GSM3415027', 'GSM3415028', 'GSM3415029', 'GSM3415030', 'GSM3415031', 'GSM3415032', 'GSM3415033', 'GSM3415034', 'GSM3415035']
Initial linked data shape: (36, 22345)
Linked data columns: [0, 1, 'A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS']...
Using trait column name: 0


Linked data shape after handling missing values: (36, 22345)
For the feature '0', the least common label is '1.0' with 17 occurrences. This represents 47.22% of the dataset.
The distribution of the feature '0' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Coronary_artery_disease/cohort_info.json


Linked data saved to ../../output/preprocess/Coronary_artery_disease/GSE120774.csv
